In [ ]:
!pip install -qU langchain-community langchain langchain-groq
!wget https://github.com/FelipeAugus/agente-com-langchain-ipynb/raw/main/levantamento-de-vulnerabilidades/prompt.md -O prompt.md

import os
import requests
from google.colab import userdata
from typing import List
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

groq_api = userdata.get("GROQ_API_KEY") # https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = groq_api


--2025-08-12 00:10:58--  https://github.com/FelipeAugus/agente-para-analise-de-vulnerabilidades-py/raw/main/prompt.md
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FelipeAugus/agente-para-analise-de-vulnerabilidades-py/main/prompt.md [following]
--2025-08-12 00:10:59--  https://raw.githubusercontent.com/FelipeAugus/agente-para-analise-de-vulnerabilidades-py/main/prompt.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3065 (3.0K) [text/plain]
Saving to: ‘prompt.md’

prompt.md           100%[===================>]   2.99K  --.-KB/s    in 0s      

2025-08-12 00:10:59 (44.4 MB/s) - ‘prompt.md’ s

In [ ]:
response = requests.get("https://api.groq.com/openai/v1/models", headers={"Authorization": f"Bearer {groq_api}"})
data = response.json()
print("Modelos disponiveis do Groq")
for model in data.get("data", []): print(model["id"])


llama-3.1-8b-instant
deepseek-r1-distill-llama-70b
compound-beta-mini
compound-beta
gemma2-9b-it
meta-llama/llama-prompt-guard-2-86m
openai/gpt-oss-120b
llama3-8b-8192
whisper-large-v3
allam-2-7b
playai-tts-arabic
llama3-70b-8192
llama-3.3-70b-versatile
meta-llama/llama-guard-4-12b
openai/gpt-oss-20b
whisper-large-v3-turbo
distil-whisper-large-v3-en
qwen/qwen3-32b
meta-llama/llama-4-maverick-17b-128e-instruct
meta-llama/llama-prompt-guard-2-22m
meta-llama/llama-4-scout-17b-16e-instruct
playai-tts
moonshotai/kimi-k2-instruct


In [ ]:
model_id = "deepseek-r1-distill-llama-70b"
llm = ChatGroq(model=model_id, temperature=0)

class ThreatItem(BaseModel):
    """Item do modelo de ameaças."""
    threat_type: str = Field(...,description="Tipo de ameaça (por exemplo, STRIDE: Spoofing, Tampering, etc.)")
    scenario: str = Field(..., description="Descrição do cenário de ameaça.")
    potential_impact: str = Field(..., description="Impacto potencial da ameaça.")

class Threats(BaseModel):
    """Saída contendo o modelo de ameaças e sugestões de melhoria."""
    threat_model: List[ThreatItem] = Field(..., description="Lista de ameaças identificadas.")
    improvement_suggestions: List[str] = Field(..., description="Sugestões para melhorar o modelo de ameaças.")

class SearchThreatAgent():
    def __init__(self):
        self.structured_llm_threats = llm.with_structured_output(Threats,
    include_raw=False,
    method="json_mode")
        self.system_msg = ""
        with open("prompt.md", "r") as f: self.system_msg = f.read()

        self.user_msg = PromptTemplate.from_template("""TIPO DE APLICAÇÃO: {tipo_aplicacao}
        MÉTODOS DE AUTENTICAÇÃO: {autenticacao}
        EXPOSTA NA INTERNET: {acesso_internet}
        DADOS SENSÍVEIS: {dados_sensiveis}
        DESCRIÇÃO DA APLICAÇÃO: {descricao_aplicacao}
        """)

    def search_threats(self, tipo_aplicacao: str, autenticacao: str, acesso_internet: str, dados_sensiveis: str, descricao_aplicacao: str) -> Threats:
        threats = self.structured_llm_threats.invoke([
            SystemMessage(content=self.system_msg),
            HumanMessage(content=self.user_msg.format(tipo_aplicacao=tipo_aplicacao, autenticacao=autenticacao, acesso_internet=acesso_internet, dados_sensiveis=dados_sensiveis, descricao_aplicacao=descricao_aplicacao))
        ])
        return threats

agente = SearchThreatAgent()

In [ ]:
ameacas = agente.search_threats(
    tipo_aplicacao="Web",
    autenticacao="EntraID",
    acesso_internet="Sim",
    dados_sensiveis="Sim",
    descricao_aplicacao="Uma aplicação para salvar contratos jurificos de clientes em uma base SQL."
)
print(ameacas)

threat_model=[ThreatItem(threat_type='Spoofing', scenario='Um atacante pode realizar um ataque de phishing contra os clientes, enviando emails fraudulentos que se parecem com comunicações oficiais da aplicação, visando roubar credenciais de acesso.', potential_impact='Perda de credenciais de clientes, permitindo acesso não autorizado à aplicação e aos contratos armazenados.'), ThreatItem(threat_type='Spoofing', scenario='Um atacante pode se passar por um cliente legítimo, utilizando uma sessão roubada ou credenciais comprometidas, para acessar ou modificar contratos.', potential_impact='Acesso não autorizado e potencial alteração ou roubo de contratos jurídicos.'), ThreatItem(threat_type='Spoofing', scenario='Um atacante pode criar um site falso que imita a interface da aplicação, enganando os usuários a inserir suas credenciais.', potential_impact='Credenciais de usuários comprometidas, permitindo acesso não autorizado à aplicação.'), ThreatItem(threat_type='Tampering', scenario='Um a

In [ ]:
print(f'----- AMEACAS -----')
print(f'Quantidade de ameaças identificadas: {len(ameacas.threat_model)}')
for threat in ameacas.threat_model:
    print('*'*25)
    print(f'Tipo de Ameaça: {threat.threat_type}')
    print(f'Cenário de exemplo: {threat.scenario}')
    print(f'Impactos: {threat.potential_impact}')

Quantidade de ameaças identificadas: 15
----- AMEACAS -----
*************************
Tipo de Ameaça: Spoofing
Cenário de exemplo: Um atacante pode realizar um ataque de phishing contra os clientes, enviando emails fraudulentos que se parecem com comunicações oficiais da aplicação, visando roubar credenciais de acesso.
Impactos: Perda de credenciais de clientes, permitindo acesso não autorizado à aplicação e aos contratos armazenados.
*************************
Tipo de Ameaça: Spoofing
Cenário de exemplo: Um atacante pode se passar por um cliente legítimo, utilizando uma sessão roubada ou credenciais comprometidas, para acessar ou modificar contratos.
Impactos: Acesso não autorizado e potencial alteração ou roubo de contratos jurídicos.
*************************
Tipo de Ameaça: Spoofing
Cenário de exemplo: Um atacante pode criar um site falso que imita a interface da aplicação, enganando os usuários a inserir suas credenciais.
Impactos: Credenciais de usuários comprometidas, permitindo 

In [ ]:
print(f'----- MELHORIAS SUGERIDAS -----')
print(f'Quantidade de melhorias sugeridas: {len(ameacas.improvement_suggestions)}')
for i, s in enumerate(ameacas.improvement_suggestions): print(f'{i+1}. {s}')

----- MELHORIAS SUGERIDAS -----
Quantidade de melhorias sugeridas: 8
1. Por favor, forneça mais detalhes sobre o fluxo de autenticação entre os componentes para permitir uma análise melhor de possíveis falhas de autenticação.
2. Considere adicionar informações sobre como os dados sensíveis são armazenados e transmitidos para permitir uma análise mais precisa de exposição de dados.
3. Detalhes sobre a arquitetura da aplicação, incluindo os componentes envolvidos no armazenamento e gerenciamento de contratos, seriam úteis.
4. Especificações sobre os mecanismos de validação de entrada e prevenção contra injeção SQL seriam importantes para identificar vulnerabilidades de tampering.
5. Informações sobre os protocolos de criptografia utilizados para proteger os dados em trânsito e em repouso ajudariam a avaliar melhor os riscos de exposição de informações.
6. Detalhes sobre os mecanismos de auditoria e logging seriam úteis para avaliar a capacidade de detectar e responder a incidentes de seg